# 1. filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 조건을 만족하는 Group의 데이터(행)들을 조회할 때 사용한다. 주로 조건은 group별 집계결과를 이용한다.
    1. 함수에 group별 DataFrame을 argument로 전달한다.
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교해서 반환한다.(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환한다.
- 매개변수
    - **func**: filtering 조건을 구현한 함수 객체
        - 첫번째 매개변수로 Group으로 묶인 DataFrame을 받는다.
        - bool type 값을 반환한다. 매개변수로 받은 DataFrame이 특정 조건을 만족하는지 여부를 반환한다.
    - **dropna=True**
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop시킨다(기본값). False로 설정하면 NA 처리해서 반환한다.
    - **\*args, \*\*kwargs**: filter 함수의 두번째부터  선언된 매개변수에 전달할 argument 값들을 가변인자로 전달한다.

In [1]:
import numpy as np
import pandas as pd

# cnt1 값의 범위: 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [5]:
# func
# filter에서 사용할 함수를 만든다.
# cnt1의 평균이 20 이상인 과일 data들을 조회하는 함수이다.

# 03. 에서 배웠던 내용을 적용하면 다음과 같다.
# 먼저 과일별 cnt1의 평균을 구한다.
result = df.groupby("fruits")["cnt1"].mean()
result[ result >= 20 ]

fruits
귤     24.0
딸기    34.2
Name: cnt1, dtype: float64

In [ ]:
# 나는 "과일별 cnt1 값의 평균이 20이상이다"라는 조건을 만족하는 과일 name과 과일별 data 값을 받기를 원한다.
# 하지만 위 경우는 과일별 cnt1 평균 값을 반환해준다.
# 이때 filter() 함수를 이용한다.

In [9]:
# filter()를 사용하기 위해 parameter로 사용될 함수를 먼저 선언한다.

def check_mean(df):
    """
    df: DataFrame - filter에 사용하면 group별 DataFrame이 하나씩 넘어온다.
        ex) 사과 group의 DataFrame -> 귤 group의 DataFrame -> 배 group의 DataFrame -> 딸기 group의 DataFrame
    """
    return df["cnt1"].mean() >= 20

In [10]:
df.groupby("fruits").filter(check_mean)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [11]:
# 위 결과처럼 "과일별 cnt1 값의 평균이 20이상이다"라는 조건을 만족하는 과일 name과 과일별 data 값을 DataFrame으로 반환한다.
# 즉 집계 함수를 check하는 함수를 filter() 함수에 parameter로 넣는 것이다.

# 위의 함수 check_mean은 상당히 간단한 함수이다. 그래서 위를 좀 더 쉽게 하려면 lambda expression을 이용하면 된다.

In [12]:
df.groupby("fruits").filter(lambda df: df["cnt1"].mean() >= 20)

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [15]:
# func - parameter가 있는 함수
# column name, threshold(임계점) 값을 parameter로 받는다.
# 이때 주의할 점은 첫 번째 parameter는 어떤 경우에서든 df가 온다는 것이다. 이는 DataFrame을 받는다.
def check_mean(df: pd.DataFrame, column_name: str, threshold: int):
    return df[column_name].mean() >= threshold

In [17]:
# 첫 번째 parameter에는 group별 DataFrame이 전달된다. 이 과정은 filter() 함수에 의해 자동으로 이뤄진다.
# 두 번째 parameter부터는 값을 직접 넘겨줘야 한다. 이때 var args를 이용해 넘겨준다.

df.groupby("fruits").filter(check_mean, column_name = "cnt1", threshold = 30)

,fruits,cnt1,cnt2
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [20]:
# parameter에 다른 값을 입력해 사용해본다.

df.groupby("fruits").filter(check_mean, column_name = "cnt2", threshold = 200)

,fruits,cnt1,cnt2
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [19]:
# dropna paramter를 입력해본다.
# NaN 값을 DataFrame에 넣어 기존 DataFrame과 크기를 맞춰준다.

df.groupby("fruits").filter(check_mean, dropna = False, column_name = "cnt2", threshold = 100)

,fruits,cnt1,cnt2
0,사과,10.0,100.0
1,사과,12.0,103.0
2,사과,13.0,107.0
3,사과,11.0,107.0
4,사과,12.0,101.0
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [ ]:
# 즉 filter는 특정 집계 조건을 만족하는 group의 column을 보기 위해 사용하는 것이다.

# 2. transform
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy.transform(func, *args)`
    - 함수(func)에 열의 값들을 group 별로 전달 한다. 함수는 그 값을 받아 통계량을 구해 반환한다. 반환된 통계량으로 원래 값들을 변경한 Series를 반환한다. 여러 컬럼에 대해 처리할 경우 DataFrame을 반환한다.
    - func: 함수객체
        - 매개변수
            - 그룹별 컬럼값들을 받을 변수 선언
        - return
            - 계산한 통계량.
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달한다.
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달한다.
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성할 수 있다.
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용할 수있다.

## 2.1. 원본에 통계치 붙여서 비교하기

In [21]:
# transform() 사용 방법
# DataFrameGroupBy에 transform을 적용하면 group별 column의 값을 parameter func에 전달해준다.
# 그리고 parameter func는 계산한 결과를 반환한다.


df.groupby("fruits").transform("mean")

,cnt1,cnt2
0,11.6,103.6
1,11.6,103.6
2,11.6,103.6
3,11.6,103.6
4,11.6,103.6
5,24.0,54.8
6,24.0,54.8
7,24.0,54.8
8,24.0,54.8
9,24.0,54.8


In [ ]:
# 실행 결과와 같이 똑같은 크기의 집계 결과를 반환한다.

In [23]:
df.groupby("fruits")["cnt1"].transform("mean")

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [ ]:
# transform()을 어떻게 사용할 수 있을까.

In [ ]:
# transform()은 원본 DataFrame df에 tramsform()을 이용해 구한 통계치를 삽입해 하나의 DataFrame으로 만들 때 유용하다.
# column을 삽입하는 방법은 다음과 같다.
# df.insert(삽입할 column 순번, column name, 값)
# 이때 주의할 점은 insert는 기본이 inplace = True라는 것이다. 즉 원본이 바뀐다.

In [27]:
result = df.groupby("fruits").transform("mean")
df.insert(2, "cnt1 평균", result["cnt1"])


In [28]:
df.insert(4, "cnt2 평균", result["cnt2"])

In [29]:
df

,fruits,cnt1,cnt1 평균,cnt2,cnt2 평균
0,사과,10,11.6,100,103.6
1,사과,12,11.6,103,103.6
2,사과,13,11.6,107,103.6
3,사과,11,11.6,107,103.6
4,사과,12,11.6,101,103.6
5,귤,21,24.0,51,54.8
6,귤,22,24.0,57,54.8
7,귤,27,24.0,58,54.8
8,귤,24,24.0,57,54.8
9,귤,26,24.0,51,54.8


In [31]:
# 다른 경우를 살펴보자.

In [1]:
import numpy as np
import pandas as pd

# cnt1 값의 범위: 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)

In [2]:
# sample() 함수를 사용해보자. sample() 함수는 전체 DataFrame으로부터 random하게 row를 추출해준다.
# 이때 parameter n은 추출할 row의 개수이다.
df.sample(n = 3)

,fruits,cnt1,cnt2
8,귤,24,57
19,딸기,28,204
13,배,3,7


In [3]:
# parameter frac는 비율을 뜻한다. 전체 DataFrame에서 추출하고자 하는 비율을 입력하면 된다. (0 ~ 1)
# 이때 ramdom하게 추출한다.
df.sample(frac = 0.3)

,fruits,cnt1,cnt2
12,배,8,5
10,배,7,9
17,딸기,37,213
15,딸기,30,208
4,사과,12,101
7,귤,27,58


In [4]:
# 그래서 DataFrame의 순서를 random하게 섞기 위해서는 frac = 1을 입력하면 된다.
df.sample(frac = 1)

,fruits,cnt1,cnt2
16,딸기,35,217
13,배,3,7
8,귤,24,57
2,사과,13,107
18,딸기,41,206
12,배,8,5
11,배,7,9
6,귤,22,57
19,딸기,28,204
1,사과,12,103


In [5]:
df = df.sample(frac = 1)

# 위 코드 실행 결과 index 순번이 뒤죽박죽이다. 이때 다음을 실행하면 된다.
df.reset_index(drop = True, inplace = True)

In [6]:
# 이제 다음 코드를 실행해보자. 앞선 tramsform() 사용 예시는 과일끼리 붙어 있었다.
# 하지만 이번에는 순서가 random하다. 이번 경우에도 transform()을 이용해 각 과일별 평균을 구하고 그 column을 추가해보자.

df.groupby("fruits")["cnt1"].transform("mean")

0     11.6
1     24.0
2      5.4
3     24.0
4      5.4
5     11.6
6      5.4
7     34.2
8     34.2
9     11.6
10     5.4
11    11.6
12     5.4
13    34.2
14    11.6
15    34.2
16    34.2
17    24.0
18    24.0
19    24.0
Name: cnt1, dtype: float64

In [7]:
result = df.groupby("fruits").transform("mean")
df.insert(2, "해당 과일의 cnt1 평균", result["cnt1"])
df.insert(4, "해당 과일의 cnt2 평균", result["cnt2"])
df

,fruits,cnt1,해당 과일의 cnt1 평균,cnt2,해당 과일의 cnt2 평균
0,사과,11,11.6,107,103.6
1,귤,26,24.0,51,54.8
2,배,2,5.4,7,7.4
3,귤,22,24.0,57,54.8
4,배,7,5.4,9,7.4
5,사과,10,11.6,100,103.6
6,배,7,5.4,9,7.4
7,딸기,28,34.2,204,209.6
8,딸기,41,34.2,206,209.6
9,사과,13,11.6,107,103.6


## 2.2. NaN 처리
- transform이용해서 결측치를 같은 과일별 평균값으로 변환
    - 전체 평균보다 좀더 정확할 수 있다.

In [ ]:
# 먼저 numpy를 이용해 NaN이 있는 Series를 만들자.

In [13]:
import numpy as np
s = pd.Series([1, np.nan, 2, np.nan])
s

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [14]:
# NaN을 다른 값으로 변경하는 방법: fillna(변경하고자 하는 값)
# 원본은 바뀌지 않는다.
s.fillna(20)

0     1.0
1    20.0
2     2.0
3    20.0
dtype: float64

In [15]:
# fillna는 Series 역시 값으로 받을 수 있다. 그 실행은 다음과 같다.
# s.fillna(s와 동일한 크기의 series): NaN은 NaN과 같은 index name의 값으로 변경하고, NaN이 아닌 것들은 그대로 유지한다.
s.fillna(pd.Series([100, 200, 300, 400]))

0      1.0
1    200.0
2      2.0
3    400.0
dtype: float64

In [ ]:
# 실행 결과 NaN 값이 바뀌었다. 이 역시 원본은 바뀌지 않는다.

In [18]:
# 다른 예시를 보자. 아래처럼 과일 DataFrame의 cnt2 column에 NaN을 몇 개 추가해주자.
df = pd.DataFrame(data)
df.loc[[0, 1, 5, 6, 10, 11, 15, 16], "cnt2"] = np.nan
df

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,NaN
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [19]:
# NaN을 없애는 방법: dropna()
# DataFrame에서 dorpna()를 하게 되면 NaN을 가지고 있는 row가 없어진다. 이는 axis 설정을 통해 column을 없앨 수도 있다.
df.dropna()

,fruits,cnt1,cnt2
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0
12,배,8,5.0
13,배,3,7.0
14,배,2,7.0
17,딸기,37,213.0


In [20]:
df.dropna(axis = 1)

,fruits,cnt1
0,사과,10
1,사과,12
2,사과,13
3,사과,11
4,사과,12
5,귤,21
6,귤,22
7,귤,27
8,귤,24
9,귤,26


In [ ]:
# 실행 결과처럼 NaN을 가지고 있는 row 또는 column이 제거되었다.

In [27]:
# 보통 NaN은 해당 column의 평균 값으로 채운다. 우리도 cnt2의 Nan 값을 모두 cnt2의 평균으로 바꿔보자.

# cnt2의 평균을 구한다.
cnt2_mean = df["cnt2"].mean()
# cnt2의 평균을 cnt2에 대입한다.
df["cnt2"] = df["cnt2"].fillna(round(cnt2_mean))
# 결과를 확인한다.
df

,fruits,cnt1,cnt2
0,사과,10,94.0
1,사과,12,94.0
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,94.0
6,귤,22,94.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [28]:
# 결과처럼 cnt2의 NaN 값이 모두 cnt2의 평균으로 바뀌었다.
# 하지만 이는 논리적이지 못한 판단이다. 왜냐하면 과일 group별로 cnt2 값의 차이가 큰 group가 있기 때문이다. 직접 확인해보라.
# 이 실행은 바뀐 NaN 값이 틀렸을 가능성이 아주 높다.
# 그래서 전체 cnt2 평균으로 NaN 값을 대체하는 것이 아니라 각 과일 group별 평균으로 대체하는 것이 좋다.
# 이때 우리는 transform()을 사용하면 쉽게 구현할 수 있다.

In [33]:
# NaN이 있는 DataFrame 생성
df = pd.DataFrame(data)
df.loc[[0, 1, 5, 6, 10, 11, 15, 16], "cnt2"] = np.nan

df["cnt2"] = df["cnt2"].fillna( round( df.groupby("fruits")["cnt2"].transform("mean") ) )
df

,fruits,cnt1,cnt2
0,사과,10,105.0
1,사과,12,105.0
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,55.0
6,귤,22,55.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [ ]:
# 이번 실행을 통해 group별 비슷한 값이 NaN에 대입되었다. 이것이 통계적으로 훨씬 더 논리적이라고 할 수 있다.

<b style='font-size:2em'>TODO </b>

In [2]:
# 1.  data/diamonds.csv 조회

import pandas as pd
    
dia = pd.read_csv("data/diamonds.csv")


In [57]:
# 2.  cut 별 평균 가격이 4000 이상인 diamond 데이터들 조회 

result = dia.groupby("cut")["price"].transform("mean")
result[ result >= 4000 ]

1        4584.257704
3        4584.257704
8        4358.757764
12       4584.257704
14       4584.257704
            ...     
53928    4584.257704
53930    4584.257704
53931    4584.257704
53934    4584.257704
53938    4584.257704
Name: price, Length: 15401, dtype: float64

In [3]:
dia.groupby("cut").filter(lambda x : x["price"].mean() >= 4000)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
14,0.20,Premium,E,SI2,60.2,62.0,345,3.79,3.75,2.27
...,...,...,...,...,...,...,...,...,...,...
53928,0.79,Premium,E,SI2,61.4,58.0,2756,6.03,5.96,3.68
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53934,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58


In [5]:
# 3. color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 모든 diamond 데이터들 조회

def max_min_diff(columns):
    diff = columns.max() - columns.min()
    if diff >= 2 and diff < 3:
        return diff

In [7]:
result = dia.groupby("color")["carat"].transform(max_min_diff)
result

C:\Users\8roke\AppData\Local\Temp\ipykernel_1436\4211864239.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = dia.groupby("color")["carat"].transform(max_min_diff)


0        2.85
1        2.85
2        2.85
3         NaN
4         NaN
         ... 
53935     NaN
53936     NaN
53937     NaN
53938     NaN
53939     NaN
Name: carat, Length: 53940, dtype: float64

In [8]:
def carat_min_max_diff(df):
    diff = df["carat"].max() - df["carat"].min()
    return diff >= 2 and diff < 3

In [9]:
dia.groupby("color").filter(carat_min_max_diff)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
...,...,...,...,...,...,...,...,...,...,...
53929,0.71,Ideal,G,VS1,61.4,56.0,2756,5.76,5.73,3.53
53930,0.71,Premium,E,SI1,60.5,55.0,2756,5.79,5.74,3.49
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53932,0.70,Very Good,E,VS2,60.5,59.0,2757,5.71,5.76,3.47


In [10]:
# 4. clarity 별 평균 가격 컬럼을 DataFrame에 추가.


mean_price = dia.groupby("clarity")["price"].transform("mean")
dia.insert(7, "average price", mean_price)
dia

,carat,cut,color,clarity,depth,table,price,average price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,5063.028606,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3996.001148,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,3839.455391,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,3924.989395,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,5063.028606,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,3996.001148,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,3996.001148,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,3996.001148,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,5063.028606,6.15,6.12,3.74


# 3. pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드.    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼들을 행과 열로 위치시키고 집계값을 값으로 보여준다.    
역할은 `groupby()`를 이용한 집계와 같은데 **여러개 컬럼을 기준으로 groupby 를 할 경우 집계결과를 읽는 것이 더 편하다.(가독성이 좋다)**

> pivot() 함수와 역할이 다르다.   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수.

- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`
- **매개변수**
    - **index**
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - **columns**
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 된다.)
    - **values**
        - 문자열 또는 리스트. 집계할 대상 컬럼들
    - **aggfunc**
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략시): 평균을 구한다. (mean이 기본값)
    - **fill_value, dropna**
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - **margins/margins_name**
        - margin: boolean(기본: False). 총집계결과를 만들지 여부.
        - margin_name: margin의 이름 문자열로 지정 (생략시 All)

In [35]:
flights = pd.read_csv("data/flights.csv")

In [36]:
flights.groupby(["ORG_AIR", "AIRLINE"])["DEP_DELAY"].mean()

ORG_AIR  AIRLINE
ATL      AA          7.995652
         AS         15.307692
         DL          7.616505
         EV         10.140896
         F9         21.574324
                      ...    
SFO      OO         11.135827
         UA         13.858655
         US          5.470588
         VX          8.144654
         WN         11.185714
Name: DEP_DELAY, Length: 114, dtype: float64

In [ ]:
# 위와 같은 실행을 할 경우 multi index로 인해 row의 수가 너무 많아져 가독성이 떨어진다.
# 이를 해결해 주는 것이 pivot_table()이다. group를 여러 개 설정했을 경우 사용하면 편리하다.

In [ ]:
# pivot_table() 사용 방법


## 3.1. 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [37]:
flights.pivot_table(index = "AIRLINE", values = "AIR_TIME", aggfunc = "mean")

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [39]:
flights.pivot_table(columns = "AIRLINE", values = "AIR_TIME", aggfunc = "mean")

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
AIR_TIME,144.259404,147.845052,209.412963,115.334187,68.964016,127.592337,338.288288,61.318346,135.736878,76.010272,155.650521,147.686755,154.864097,107.005897


In [40]:
flights.pivot_table(index = "AIRLINE", values = "AIR_TIME", aggfunc = "mean", margins = True)

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


In [41]:
flights.pivot_table(index = "AIRLINE", values = "AIR_TIME", aggfunc = "mean", margins = True)

,AIR_TIME
AIRLINE,
AA,144.259404
AS,147.845052
B6,209.412963
DL,115.334187
EV,68.964016
F9,127.592337
HA,338.288288
MQ,61.318346
NK,135.736878


## 3.2. 2개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 구한다.)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [43]:
# groupby를 이용하는 방법
flights.groupby(["AIRLINE", "ORG_AIR"])["CANCELLED"].sum()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [42]:
# pivot_table()을 이용하는 방법
flights.pivot_table(index = "AIRLINE", columns = "ORG_AIR", values = "CANCELLED", aggfunc = "sum")

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B6,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
EV,18.0,6.0,27.0,36.0,NaN,NaN,6.0,53.0,0.0,NaN
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0
HA,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
MQ,5.0,NaN,62.0,0.0,NaN,0.0,0.0,85.0,NaN,NaN
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,NaN


In [ ]:
# 비교 실행 결과 가독성이 훨씬 좋다.

In [45]:
flights.pivot_table(index = "AIRLINE", columns = "ORG_AIR", values = "CANCELLED", aggfunc = "sum", fill_value = 10, margins = True)
# NaN을 대체한 값은 margin을 계산할 때 집계되지 않는다.

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,All
AIRLINE,,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2,154
AS,0,0,0,0,0,0,0,0,0,0,0
B6,10,0,0,10,0,0,10,0,0,1,1
DL,28,1,0,0,1,1,4,0,1,2,38
EV,18,6,27,36,10,10,6,53,0,10,146
F9,0,2,1,0,1,1,1,4,0,0,10
HA,10,10,10,10,0,0,10,10,0,0,0
MQ,5,10,62,0,10,0,0,85,10,10,152
NK,1,1,6,0,1,1,3,10,2,10,25


## 3.3. 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [46]:
# groupby를 이용하는 방법
flights.groupby(["AIRLINE", "MONTH", "ORG_AIR"])["CANCELLED"].sum()

AIRLINE  MONTH  ORG_AIR
AA       1      ATL        0
                DEN        0
                DFW        8
                IAH        0
                LAS        0
                          ..
WN       12     LAS        1
                LAX        2
                MSP        0
                PHX        0
                SFO        0
Name: CANCELLED, Length: 1133, dtype: int64

In [47]:
# pivot_table을 이용하는 방법
flights.pivot_table(index = ["AIRLINE", "ORG_AIR"], columns = "MONTH", values = "CANCELLED", aggfunc = "sum")

MONTH             1     2     3    4    5    6    7    8    9    11   12
AIRLINE ORG_AIR                                                         
AA      ATL      0.0   2.0   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
        DEN      0.0   1.0   0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  1.0
        DFW      8.0  33.0  13.0  4.0  8.0  7.0  1.0  2.0  1.0  3.0  6.0
        IAH      0.0   1.0   0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
        LAS      0.0   2.0   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
...              ...   ...   ...  ...  ...  ...  ...  ...  ...  ...  ...
WN      LAS      1.0   1.0   1.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0
        LAX      3.0   2.0   3.0  2.0  1.0  0.0  9.0  4.0  3.0  3.0  2.0
        MSP      1.0   0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
        PHX      0.0   2.0   1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
        SFO      4.0   5.0   0.0  2.0  2.0  6.0  0.0  2.0  2.0  2.0  0.0

[114 rows x 11 columns]

## 3.4. 3개 이상의 컬럼을 grouping해서 집계 2
- 항공사/월/출발공항코드 별 최대/최소 연착시간
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - ARR_DELAY: 연착시간
- 집계 : min, max    

In [52]:
# groupby
flights.groupby(["AIRLINE", "MONTH", "ORG_AIR"])["ARR_DELAY"].agg(["min", "max"])

min    max
AIRLINE MONTH ORG_AIR             
AA      1     ATL     -27.0   26.0
              DEN     -13.0   78.0
              DFW     -39.0  287.0
              IAH     -23.0   63.0
              LAS     -32.0  732.0
...                     ...    ...
WN      12    LAS     -52.0   96.0
              LAX     -30.0  493.0
              MSP     -23.0   90.0
              PHX     -30.0  254.0
              SFO     -20.0  215.0

[1133 rows x 2 columns]

In [54]:
result = flights.pivot_table(index = "MONTH", columns = ["AIRLINE", "ORG_AIR"], values = "ARR_DELAY", 
                     aggfunc = ["min", "max"])
result.to_excel("saved_data/pivot_table_3_4.xlsx")

# 4. apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리가 가능하다.

- DataFrame.apply(함수, axis=0, args=(), \*\*kwarg)
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달한다.
    - 매개변수
        - **함수**: DataFrame의 행들 또는 열들을 전달할 함수
        - **axis**: **0-컬럼(열)을 전달, 1-행을 전달 (기본값 0)**
        - **args**: 함수에 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 행/열 이외에 전달할 매개변수를 키워드 인자로 전달
- Series.apply(함수, args=(), \*\*kwarg)
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달한다.
    - 매개변수
        - **함수**: Series의 원소들을 전달할 함수
        - **args**: 함수에 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 원소 이외에 전달할 매개변수를 키워드 인자로 전달

In [20]:
# apply() 사용 방법


# 먼저 0 ~ 23까지 1씩 증가하는 정수로 구성된 1차원 배열을 만들어보자. -> np.arange(24)
# 그리고 이 1차원 배열을 6 X 4 형태의 2차원 배열로 변환하자. -> reshape(6, 4)
# 이때 element의 개수가 같아야함에 주의한다.

import numpy as np
import pandas as pd

a = np.arange(24).reshape(6, 4)

# 이렇게 만든 2차원 배열을 DataFrame으로 만들자.
df = pd.DataFrame(a, columns = ["no1", "no2", "no3", "no4"])

In [39]:
# apply()에 사용할 간단한 사용자 정의 함수를 하나 만든다.

def func(x):
    return x * 100

In [40]:
# DataFrame.apply()

df.apply(func)

,no1,no2,no3,no4
0,0,100,200,300
1,400,500,600,700
2,800,900,1000,1100
3,1200,1300,1400,1500
4,1600,1700,1800,1900
5,2000,2100,2200,2300


In [41]:
# Series.apply()

df["no1"].apply(func)

0       0
1     400
2     800
3    1200
4    1600
5    2000
Name: no1, dtype: int64

In [ ]:
# 보이는 결과와 같이 DataFrame에 apply를 사용하면 Series(row 또는 column)별로 계산을 해 그 결과를 DataFrame으로 반환한다.
# Series에 apply를 사용하면 element별로 계산을 해 그 결과를 Series로 반환한다.

In [27]:
# apply()에 사용할 사용자 정의 함수를 하나 만든다.
def func(x):
    """
    x
    - DataFrame.apply()에 사용될 때 - DataFrame의 Series가 하나씩 전달된다.
        axis = 0일 때 column 전달
        axis = 1일 때 row 전달
    - Series.apply()에 사용될 때 - Series의 element가 하나씩 전달된다.
    """
    if x.name in ["no1", "no3"]:
        return x * 100
    else:
        return x / 100

In [28]:
df

,no1,no2,no3,no4
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
5,20,21,22,23


In [29]:
df.apply(func)

,no1,no2,no3,no4
0,0,0.01,200,0.03
1,400,0.05,600,0.07
2,800,0.09,1000,0.11
3,1200,0.13,1400,0.15
4,1600,0.17,1800,0.19
5,2000,0.21,2200,0.23


In [ ]:
# axis = 0이 기본값이다. 그래서 column이 하나씩 전달되어 계산되었다.

In [ ]:
# 참고 .name
# dataframe으로부터 불러온 Series의 name을 알 수 있다.
# column을 Series로 불러왔을 때는 Series의 column name을 알 수 있다.
# row를 Series로 불러왔을 때는 Series의 row name을 알 수 있다. 

In [30]:
# aplly()에 lambda expression을 이용해보자. 아래와 같은 코드를 작성하면 각 column 별로 계산을 해 처리 결과를 Series로 반환한다.

df.apply(lambda x : x.max()-x.min())

no1    20
no2    20
no3    20
no4    20
dtype: int32

In [38]:
df.apply(lambda x : x.mean() - x.median())

no1    0.0
no2    0.0
no3    0.0
no4    0.0
dtype: float64

# 5. cut()/qcut() - 연속형(실수)을 범주형으로 변환
- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶는다.
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - **x**: 범주형으로 바꿀 대상. 1차원 배열형태(Series, 리스트, ndarray)의 자료구조
        - **bins**: 범주로 나눌때의 기준값(구간경계)들을 리스트로 묶어서 전달한다.
        - **right**: 구간경계의 오른쪽(True-기본)을 포함할지 왼쪽(False)을 포함할지
        - **labels**: 각 구간(범주)의 label을 리스트로 전달
            - 생략하면 범위를 범주명으로 사용한다. (ex: (10, 20], ()-포함안함, []-포함)
- qcut() :  대상배열의 최대값 ~ 최소값을 지정한 개수의 동등한 size(**원소의개수**)가 되도록 나눈다.
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - **x**: 나눌 대상. 1차원 배열형태의 자료구조
        - **q**: 나눌 개수
        - **labels**: 각 구간(범주)의 label을 리스트로 전달

In [47]:
# cut() / qcut() 사용 방법

# DataFrame 생성
# ramdom한 data를 생성한다.

# 항상 동일한 난수를 생성하기 위해 seed 값을 지정한다.
np.random.seed(0)

# age는 1 ~ 99 사이의 난수 30개를 가지고 있는 배열이다. 중복을 허용한다.
age = np.random.randint(1, 100, size = 30)

# tall은 평균이 170이고 표준 편차가 10인 정규 분포를 따르는 난수 30개를 가지고 있는 배열이다.
# 즉 대부분의 난수가 (170-10*2) ~ (170+10*2) 사이의 수를 갖는다. 그리고 평균에 가까운 값들의 수가 많다.
# 그리고 소수점 아래 둘째 자리까지 반올림한다.
tall = np.round(np.random.normal(170, 10, size = 30), 2)

In [49]:
# DataFrame을 만든다.

df = pd.DataFrame({
    "나이": age,
    "키": tall
})
df

,나이,키
0,45,168.18
1,48,184.10
2,65,166.26
3,68,172.75
4,68,160.39
5,10,173.77
6,84,170.33
7,22,176.81
8,37,154.37
9,88,164.33


In [51]:
# 위 DataFrame은 quantitative data이다. 이를 categoricla data로 바꿔보자.

In [54]:
# 방법 1. cut() 사용 - category의 개수를 설정


# parameter x & bins
# df["나이"] data를 3등분해보자.
나이대 = pd.cut(df["나이"], bins = 3)
나이대

0     (36.333, 62.667]
1     (36.333, 62.667]
2       (62.667, 89.0]
3       (62.667, 89.0]
4       (62.667, 89.0]
5      (9.921, 36.333]
6       (62.667, 89.0]
7      (9.921, 36.333]
8     (36.333, 62.667]
9       (62.667, 89.0]
10      (62.667, 89.0]
11      (62.667, 89.0]
12      (62.667, 89.0]
13     (9.921, 36.333]
14    (36.333, 62.667]
15      (62.667, 89.0]
16    (36.333, 62.667]
17      (62.667, 89.0]
18    (36.333, 62.667]
19      (62.667, 89.0]
20      (62.667, 89.0]
21    (36.333, 62.667]
22     (9.921, 36.333]
23      (62.667, 89.0]
24      (62.667, 89.0]
25     (9.921, 36.333]
26     (9.921, 36.333]
27      (62.667, 89.0]
28      (62.667, 89.0]
29      (62.667, 89.0]
Name: 나이, dtype: category
Categories (3, interval[float64, right]): [(9.921, 36.333] < (36.333, 62.667] < (62.667, 89.0]]

In [ ]:
# 값이 9.921, 36.333, 62.667, 89를 갖는다.
# 즉 첫 번째 그룹은 9.921 ~ 36.333
# 두 번째 그룹은 36.333 ~ 62.667
# 세 번째 그룹은 62.667 ~ 89 이다.

# 반환된 data를 보면 소괄호와 대괄호로 묶여있다.
# 소괄호는 opend를 뜻하며 값을 불포함하고, 대괄호는 closed를 뜻하며 값을 포함한다.
# 예를 들어 (36.333, 62.667]는 36.333 < value <= 62.667 을 뜻하는 것이다.
# DataFrame을 보자. 나이 column의 0 row에 있는 값은 45이다.
# 그래서 category가 (36.333, 62.667]으로 나오는 것이다.
# 나이 column의 11 row에 있는 값은 89이다.
# 그래서 category가 (62.667, 89.0]으로 나온다. 즉 89라는 값을 포함하는 category가 만들어진 것이다.



In [55]:
# parameter right

# 구간 경계의 포함 방법은 right parameter를 이용한다.
나이대 = pd.cut(df["나이"], bins = 3, right = False)
나이대

0     [36.333, 62.667)
1     [36.333, 62.667)
2     [62.667, 89.079)
3     [62.667, 89.079)
4     [62.667, 89.079)
5       [10.0, 36.333)
6     [62.667, 89.079)
7       [10.0, 36.333)
8     [36.333, 62.667)
9     [62.667, 89.079)
10    [62.667, 89.079)
11    [62.667, 89.079)
12    [62.667, 89.079)
13      [10.0, 36.333)
14    [36.333, 62.667)
15    [62.667, 89.079)
16    [36.333, 62.667)
17    [62.667, 89.079)
18    [36.333, 62.667)
19    [62.667, 89.079)
20    [62.667, 89.079)
21    [36.333, 62.667)
22      [10.0, 36.333)
23    [62.667, 89.079)
24    [62.667, 89.079)
25      [10.0, 36.333)
26      [10.0, 36.333)
27    [62.667, 89.079)
28    [62.667, 89.079)
29    [62.667, 89.079)
Name: 나이, dtype: category
Categories (3, interval[float64, left]): [[10.0, 36.333) < [36.333, 62.667) < [62.667, 89.079)]

In [ ]:
# 위와 같이 category의 좌우측 값 포함 여부가 바뀌었다.

In [57]:
# parameter labels

# 위와 같은 코드를 보면 category가 숫자로 표현되어 있다. 이때 각 category의 이름을 설정할 수 있다.

나이대 = pd.cut(df["나이"], bins = 3, labels = ["나이대1", "나이대2", "나이대3"])
나이대

0     나이대2
1     나이대2
2     나이대3
3     나이대3
4     나이대3
5     나이대1
6     나이대3
7     나이대1
8     나이대2
9     나이대3
10    나이대3
11    나이대3
12    나이대3
13    나이대1
14    나이대2
15    나이대3
16    나이대2
17    나이대3
18    나이대2
19    나이대3
20    나이대3
21    나이대2
22    나이대1
23    나이대3
24    나이대3
25    나이대1
26    나이대1
27    나이대3
28    나이대3
29    나이대3
Name: 나이, dtype: category
Categories (3, object): ['나이대1' < '나이대2' < '나이대3']

In [58]:
df["나이대1"] = 나이대
df

,나이,키,나이대1
0,45,168.18,나이대2
1,48,184.10,나이대2
2,65,166.26,나이대3
3,68,172.75,나이대3
4,68,160.39,나이대3
5,10,173.77,나이대1
6,84,170.33,나이대3
7,22,176.81,나이대1
8,37,154.37,나이대2
9,88,164.33,나이대3


In [63]:
# category의 기준점을 지정하는 방법
# 0 ~ 20, 20 ~ 40, 40 ~ 60, 60 ~ 90의 category로 만들고 label을 붙여보자.

l = [0, 20, 40, 60, 90]

나이대2 = pd.cut(df["나이"], bins = l, labels = ["청소년", "청년", "장년", "노년"])
df["나이대2"] = 나이대2
df

,나이,키,나이대1,나이대2
0,45,168.18,나이대2,장년
1,48,184.10,나이대2,장년
2,65,166.26,나이대3,노년
3,68,172.75,나이대3,노년
4,68,160.39,나이대3,노년
5,10,173.77,나이대1,청소년
6,84,170.33,나이대3,노년
7,22,176.81,나이대1,청년
8,37,154.37,나이대2,청년
9,88,164.33,나이대3,노년


In [64]:
# 위와 같이 category를 설정하면 숫자들로만 구성된 data를 group으로 나눠 집계할 수 있다.
# 아래와 같이 각 나이대별 수를 구하거나 비율을 구할 수도 있다.
df["나이대2"].value_counts()

노년     17
청년      6
장년      4
청소년     3
Name: 나이대2, dtype: int64

In [66]:
df["나이대2"].value_counts(normalize = True)

노년     0.566667
청년     0.200000
장년     0.133333
청소년    0.100000
Name: 나이대2, dtype: float64

In [68]:
# 그리고 아래처럼 나이대에 따른 평균 키를 구할 수도 있다.
df.groupby("나이대2")["키"].mean()

나이대2
청소년    173.546667
청년     168.171667
장년     174.032500
노년     170.558235
Name: 키, dtype: float64

In [70]:
# 방법 2. qcut() 사용 방법 - category의 element의 개수가 갖게 설정

# parameter x & q
키대 = pd.qcut(df["키"], q = 3)
키대

0                  (166.22, 174.4]
1                  (174.4, 187.87]
2                  (166.22, 174.4]
3                  (166.22, 174.4]
4     (150.66899999999998, 166.22]
5                  (166.22, 174.4]
6                  (166.22, 174.4]
7                  (174.4, 187.87]
8     (150.66899999999998, 166.22]
9     (150.66899999999998, 166.22]
10                 (166.22, 174.4]
11                 (174.4, 187.87]
12                 (166.22, 174.4]
13                 (166.22, 174.4]
14                 (174.4, 187.87]
15                 (174.4, 187.87]
16                 (174.4, 187.87]
17                 (166.22, 174.4]
18    (150.66899999999998, 166.22]
19                 (174.4, 187.87]
20                 (174.4, 187.87]
21    (150.66899999999998, 166.22]
22                 (166.22, 174.4]
23    (150.66899999999998, 166.22]
24    (150.66899999999998, 166.22]
25                 (174.4, 187.87]
26    (150.66899999999998, 166.22]
27    (150.66899999999998, 166.22]
28                 (

In [71]:
키대.value_counts()

(150.66899999999998, 166.22]    10
(166.22, 174.4]                 10
(174.4, 187.87]                 10
Name: 키, dtype: int64

In [ ]:
# 위를 보면 각 category별로 element의 개수가 같다는 것을 알 수 있다.

In [72]:
# parameter label
키대 = pd.qcut(df["키"], q = 3, labels = ["작은키", "중간키", "큰키"])
키대

0     중간키
1      큰키
2     중간키
3     중간키
4     작은키
5     중간키
6     중간키
7      큰키
8     작은키
9     작은키
10    중간키
11     큰키
12    중간키
13    중간키
14     큰키
15     큰키
16     큰키
17    중간키
18    작은키
19     큰키
20     큰키
21    작은키
22    중간키
23    작은키
24    작은키
25     큰키
26    작은키
27    작은키
28     큰키
29    작은키
Name: 키, dtype: category
Categories (3, object): ['작은키' < '중간키' < '큰키']

In [78]:
df["키대"] = 키대

In [74]:
# 키대에 따른 나이의 평균을 구해보자.
df.groupby("키대")["나이"].mean()

키대
작은키    61.1
중간키    55.9
큰키     57.5
Name: 나이, dtype: float64

<b style='font-size:2em'>TODO </b>

In [3]:
# 1: data/diamonds.csv 를 읽어 DataFrame으로 만든다.

import pandas as pd

dia = pd.read_csv("data/diamonds.csv")
dia_df = pd.DataFrame(dia)

In [9]:
# 2: price 컬럼을 '고가', '중가', '저가' 세개의 범주값을 가지는 "price_cate" 컬럼을 생성한다.

price_cate = pd.cut( dia_df["price"], bins = 3, labels = ["저가", "중가", "고가"] )
dia_df.insert( 7, "price_cate", price_cate )

In [88]:
dia_df

,carat,cut,color,clarity,depth,table,price,price_cate,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,저가,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,저가,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,저가,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,저가,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,저가,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,저가,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,저가,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,저가,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,저가,6.15,6.12,3.74


In [89]:
dia_df["price_cate"].value_counts()

저가    43591
중가     7347
고가     3002
Name: price_cate, dtype: int64

In [ ]:
import pandas as pd

dia = pd.read_csv("data/diamonds.csv")
dia_df = pd.DataFrame(dia)

In [5]:
# 내가 원하는 category 범위로 category를 설정할 때

pd.cut(dia_df["price"], bins = [300, 1000, 10000, 20000], labels = ["저가", "중가", "고가"])

0        저가
1        저가
2        저가
3        저가
4        저가
         ..
53935    중가
53936    중가
53937    중가
53938    중가
53939    중가
Name: price, Length: 53940, dtype: category
Categories (3, object): ['저가' < '중가' < '고가']

In [7]:
# category의 element의 개수를 갖게 설정할 때

pd.qcut(dia_df["price"], q = 3, labels = ["저가", "중가", "고가"])

0        저가
1        저가
2        저가
3        저가
4        저가
         ..
53935    중가
53936    중가
53937    중가
53938    중가
53939    중가
Name: price, Length: 53940, dtype: category
Categories (3, object): ['저가' < '중가' < '고가']

In [10]:
# 3 가격대(price_cate) 별 carat의 평균을 조회

dia.groupby("price_cate")["carat"].mean()

price_cate
저가    0.627943
중가    1.362280
고가    1.885256
Name: carat, dtype: float64

In [11]:
# 4 가격대(price_cate)와 cut별 평균 가격(price)를 피봇테이블로 조회

dia.pivot_table( index = "cut", columns = "price_cate", values = "price", aggfunc = "mean" )

price_cate,저가,중가,고가
cut,,,
Fair,2976.492819,8891.044335,15176.404762
Good,2589.423161,8744.889273,15412.570213
Ideal,1991.838539,9019.390308,15422.528676
Premium,2513.585759,9045.209888,15391.247016
Very Good,2383.723178,8921.683740,15502.217323


In [12]:
# 5 cut, color, price_cate 별 carat의 평균을 피봇테이블로 조회

dia.pivot_table( index = ["cut", "color"], columns = "price_cate", values = "carat", aggfunc = "mean" )

price_cate             저가        중가        고가
cut       color                              
Fair      D      0.807122  1.339412  2.145714
          E      0.767424  1.443500  1.843333
          F      0.791801  1.564483  1.957273
          G      0.849462  1.807895  1.995000
          H      0.977699  1.796226  2.218750
          I      0.988750  1.784000  2.407778
          J      1.067717  1.985000  2.691818
Good      D      0.670725  1.277551  1.626500
          E      0.651305  1.256889  1.717097
          F      0.689913  1.298312  1.809286
          G      0.708228  1.349915  1.864048
          H      0.730387  1.523187  2.093810
          I      0.764122  1.630000  2.104643
          J      0.854693  1.695397  2.242500
Ideal     D      0.502135  1.170843  1.430909
          E      0.511158  1.198333  1.501395
          F      0.529869  1.189853  1.609551
          G      0.528951  1.193722  1.694875
          H      0.617462  1.398540  1.947143
          I      0.669981  1.533642  2.097365
          J      0.792335  1.645655  2.229880
Premium   D      0.599525  1.264649  1.636000
          E      0.591792  1.260486  1.715130
          F      0.666792  1.268342  1.705594
          G      0.629871  1.300076  1.793261
          H      0.765683  1.525855  2.069171
          I      0.794620  1.610519  2.127762
          J      0.901903  1.716263  2.236724
Very Good D      0.599320  1.226980  1.552857
          E      0.577348  1.212459  1.740141
          F      0.619161  1.193546  1.688632
          G      0.614364  1.244620  1.735664
          H      0.712640  1.489892  2.019664
          I      0.760677  1.551324  2.100703
          J      0.832754  1.707778  2.150377